<a href="https://colab.research.google.com/github/GonorAndres/Analisis_Numerico_2025-2/blob/main/Practica3/Ejercio27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Problema 27: sistema mínimo cuadrado sensible con parámetro ε

#27a

import numpy as np
import pandas as pd
from numpy.linalg import norm

# Funciones reutilizadas
def minimos_cuadrados_manual(A, b):
    AtA = A.T @ A
    Atb = A.T @ b
    return resolver_sistema_por_gauss(AtA.copy(), Atb.copy())

def resolver_sistema_por_gauss(A, b):
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)
    for i in range(n):
        for j in range(i+1, n):
            if A[i, i] == 0:
                continue
            factor = A[j, i] / A[i, i]
            A[j, i:] -= factor * A[i, i:]
            b[j] -= factor * b[i]
    x = np.zeros(n)
    for i in reversed(range(n)):
        if A[i, i] == 0:
            x[i] = 0
        else:
            x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    return x

# Probar con diferentes valores de ε
epsilons = [1e-4, 1e-8, 1e-12]
x_sols = []
for eps in epsilons:
    A_27 = np.array([
        [1, 1, 1],
        [eps, 0, 0],
        [0, eps, 0],
        [0, 0, eps]
    ])
    b_27 = np.array([1, 0, 0, 0])
    x_eps = minimos_cuadrados_manual(A_27, b_27)
    x_sols.append({
        "ε": eps,
        "x̃ (redondeado)": np.round(x_eps, 6),
        "Norma ||x̃||": norm(x_eps)
    })

df_27 = pd.DataFrame(x_sols)
df_27

,ε,x̃ (redondeado),Norma ||x̃||
0,1.000000e-04,"[0.333333, 0.333333, 0.333333]",0.57735
1,1.000000e-08,"[1.0, 0.0, 0.0]",1.00000
2,1.000000e-12,"[1.0, 0.0, 0.0]",1.00000


In [7]:
# 27b
# Reutilizamos la misma matriz A y vector b para los mismos valores de epsilon

def householder_qr(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    for k in range(n):
        x = R[k:, k]
        e1 = np.zeros_like(x)
        e1[0] = 1
        v = x + np.sign(x[0]) * norm(x) * e1
        v = v / norm(v)
        Hk = np.eye(m)
        Hk[k:, k:] -= 2.0 * np.outer(v, v)
        R = Hk @ R
        Q = Q @ Hk
    return Q, R

# Resolver sistema Ax ≈ b con QR de Householder
def resolver_qr_householder(A, b):
    Q, R = householder_qr(A)
    # Resolve R_upper_triangle * x = (Q.T * b)_top_n
    # R[:A.shape[1], :] gives the upper n x n part of R where n is A.shape[1]
    # Q.T @ b gives a vector of size m
    # We need to take the first n elements of Q.T @ b
    return np.linalg.solve(R[:A.shape[1], :], (Q.T @ b)[:A.shape[1]])

# Ejecutar para cada epsilon
x_qr_sols = []
for eps in [1e-4, 1e-8, 1e-12]:
    A = np.array([
        [1, 1, 1],
        [eps, 0, 0],
        [0, eps, 0],
        [0, 0, eps]
    ])
    b = np.array([1, 0, 0, 0])
    x_qr = resolver_qr_householder(A, b)
    x_qr_sols.append({
        "ε": eps,
        "x̃ (redondeado)": np.round(x_qr, 6),
        "Norma ||x̃||": norm(x_qr)
    })

df_27b = pd.DataFrame(x_qr_sols)
df_27b

,ε,x̃ (redondeado),Norma ||x̃||
0,1.000000e-04,"[0.333333, 0.333333, 0.333333]",0.57735
1,1.000000e-08,"[0.333333, 0.333333, 0.333333]",0.57735
2,1.000000e-12,"[0.333333, 0.333333, 0.333333]",0.57735


In [8]:
#27c

import numpy as np
import pandas as pd
from numpy.linalg import norm

def givens_rotation(a, b):
    r = np.hypot(a, b)
    if r == 0:
        return 1, 0
    c = a / r
    s = -b / r
    return c, s

def qr_givens(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    for j in range(n):
        for i in range(m-1, j, -1):
            c, s = givens_rotation(R[i-1, j], R[i, j])
            G = np.eye(m)
            G[[i-1, i], [i-1, i]] = c
            G[i-1, i] = -s
            G[i, i-1] = s
            R = G @ R
            Q = Q @ G.T
    return Q, R

def resolver_qr_givens(A, b):
    Q, R = qr_givens(A)
    Qtb = Q.T @ b
    return np.linalg.solve(R[:A.shape[1], :], Qtb[:A.shape[1]])

# Ejecutar para diferentes valores de epsilon
epsilons = [1e-4, 1e-8, 1e-12]
resultados_givens = []

for eps in epsilons:
    A = np.array([
        [1, 1, 1],
        [eps, 0, 0],
        [0, eps, 0],
        [0, 0, eps]
    ])
    b = np.array([1, 0, 0, 0])
    x_givens = resolver_qr_givens(A, b)
    resultados_givens.append({
        "ε": eps,
        "x̃ (redondeado)": np.round(x_givens, 6),
        "Norma ||x̃||": norm(x_givens)
    })

pd.DataFrame(resultados_givens)


,ε,x̃ (redondeado),Norma ||x̃||
0,1.000000e-04,"[0.333333, 0.333333, 0.333333]",0.57735
1,1.000000e-08,"[0.333333, 0.333333, 0.333333]",0.57735
2,1.000000e-12,"[0.333333, 0.333333, 0.333333]",0.57735
